<a href="https://colab.research.google.com/github/CiaraFarrellSETU/phd/blob/main/rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 84.9 MB/s eta 0:00:00


In [ ]:
import rasterio
import numpy as np
import geopandas as gpd
from rasterio.features import rasterize
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

In [ ]:
with rasterio.open("/content/RGB (1).tif") as src:
    img = src.read()  # shape: (bands, rows, cols)
    profile = src.profile
    transform = src.transform
    out_shape = (src.height, src.width)

In [ ]:
gdf = gpd.read_file('/content/Ballymore_habitat_map2018_ITM.shp')
#print(gdf.head())
#print(gdf.crs)
print(gdf.geometry.is_valid)
print(gdf.geometry.is_empty)

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
dtype: bool


In [ ]:
print(gdf.columns)

Index(['Id', 'FossittTyp', 'FossittCod', 'EUType', 'EUCode', 'Habitat',
       'ClassID', 'geometry'],
      dtype='object')


In [ ]:
print(gdf.head())

   Id                                         FossittTyp FossittCod  \
0   0                    Transition mire and quaking bog        PF3   
1   0                                 Rich fen and flush        PF1   
2   0  Rich fen and flush/Transition mire and quaking...    PF1/PF3   
3   0                                         Raised bog        PB1   
4   0                                 Rich fen and flush        PF1   

                                           EUType     EUCode  \
0               Transition mires and quaking bogs       7140   
1                                    Alkaline fen       7230   
2  Alkaline fen/Transition mires and quaking bogs  7230/7140   
3                                            None       None   
4                                    Alkaline fen       7230   

                                     Habitat  ClassID  \
0           Carex-Menyanthes transition mire        1   
1                    Juncus subnodulosus fen        2   
2  Mosaic Schoenu

In [ ]:
gdf = gdf[gdf.geometry.notnull()]
gdf = gdf[~gdf.geometry.is_empty]

# Fix invalid geometries (common trick: buffer(0))
gdf["geometry"] = gdf.buffer(0)

# Check again
print("Valid geometries:", gdf.geometry.is_valid.sum())
print("Total geometries:", len(gdf))
print("Bounds:", gdf.total_bounds)

Valid geometries: 17
Total geometries: 17
Bounds: [623748.24179765 749124.98347799 624362.67167234 749762.56444066]


In [ ]:
# Assuming there is a column named 'Habitat' in your shapefile
if "ClassID" not in gdf.columns:
    if "Habitat" in gdf.columns:
        gdf["ClassID"] = gdf["Habitat"].astype("category").cat.codes
    else:
        print("Error: 'Habitat' column not found in the shapefile attributes.")

In [ ]:
print("Raster bounds:", src.bounds)
print("Shapefile bounds:", gdf.total_bounds)

Raster bounds: BoundingBox(left=589948.95657, bottom=5927713.406020001, right=590857.04865, top=5928676.113220001)
Shapefile bounds: [623748.24179765 749124.98347799 624362.67167234 749762.56444066]


In [ ]:
print("Raster CRS:", profile["crs"])
print("Shapefile CRS:", gdf.crs)


Raster CRS: EPSG:32629
Shapefile CRS: EPSG:2157


In [ ]:
#gdf = gdf.set_crs("EPSG:32629")

In [ ]:
gdf = gpd.read_file("/content/Ballymore_habitat_map2018_ITM.shp")

# Force assign Irish Transverse Mercator (EPSG:2157)
gdf = gdf.set_crs("EPSG:2157", allow_override=True)

# Reproject to raster CRS (EPSG:32629)
gdf = gdf.to_crs("EPSG:32629")

print("Reprojected shapefile bounds:", gdf.total_bounds)

Reprojected shapefile bounds: [ 590081.50819806 5927821.7540698   590694.89681513 5928459.75002118]


In [ ]:
shapes = ((geom, value) for geom, value in zip(gdf.geometry, gdf["ClassID"]))
labels = rasterize(
    shapes=shapes,
    out_shape=out_shape,
    transform=transform,
    fill=0,
    dtype="int32"
)

print("Unique values in rasterized labels:", np.unique(labels))

Unique values in rasterized labels: [0 1 2 3 4 5 6 7 8]


In [ ]:
print("Unique values in rasterized labels:", np.unique(labels))

Unique values in rasterized labels: [0 1 2 3 4 5 6 7 8]


In [ ]:
n_bands, n_rows, n_cols = img.shape
X = img.reshape(n_bands, -1).T   # (pixels, bands)
y = labels.flatten()

In [ ]:
mask = y > 0
X = X[mask]
y = y[mask]

In [ ]:
print('Shape of X after masking', X.shape)
print('Shape of y after masking', y.shape)

Shape of X after masking (94067, 4)
Shape of y after masking (94067,)


The model won’t struggle with dimensionality (only 4 predictors).

The main challenge is avoiding overfitting while still capturing signal in the data.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
#rf = RandomForestClassifier(n_estimators=500, random_state=42, n_jobs=-1)
#rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)

In [ ]:

rf = RandomForestClassifier(
    n_estimators=350,       # more stable than default 100
    max_depth=30,         # none let trees grow fully first
    min_samples_split=5,    # reduce overfitting
    min_samples_leaf=10,     # avoid tiny leaves
    max_features="log2",    # sqrt good with few features
    class_weight="balanced",# if classes are imbalanced
    n_jobs=-1,              # use all cores
    random_state=42

)

rf.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced', max_depth=30,
                       max_features='log2', min_samples_leaf=10,
                       min_samples_split=5, n_estimators=350, n_jobs=-1,
                       random_state=42)

In [ ]:
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.30      0.16      0.21      3083
           2       0.18      0.29      0.22      1430
           3       0.54      0.51      0.53      2903
           4       0.30      0.29      0.29      2436
           5       0.48      0.36      0.42      4177
           6       0.38      0.63      0.47      1548
           7       0.22      0.21      0.22      1897
           8       0.13      0.23      0.17      1340

    accuracy                           0.34     18814
   macro avg       0.32      0.34      0.32     18814
weighted avg       0.36      0.34      0.34     18814



In [ ]:
full_pred = rf.predict(img.reshape(n_bands, -1).T)
classified = full_pred.reshape(n_rows, n_cols)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_dist = {
    "n_estimators": [200, 500, 800],
    "max_depth": [10, 20, None],
    "min_samples_leaf": [1, 5, 10],
    "max_features": ["sqrt", "log2", None]
}

rand = RandomizedSearchCV(
    RandomForestClassifier(class_weight="balanced", n_jobs=-1, random_state=42),
    param_distributions=param_dist,
    n_iter=20,  # sample 20 random combos
    cv=3,
    scoring="f1_macro",
    random_state=42
)
rand.fit(X_train, y_train)


RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(class_weight='balanced',
                                                    n_jobs=-1,
                                                    random_state=42),
                   n_iter=20,
                   param_distributions={'max_depth': [10, 20, None],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 5, 10],
                                        'n_estimators': [200, 500, 800]},
                   random_state=42, scoring='f1_macro')

In [ ]:
rf = RandomForestClassifier(
    n_estimators=200,       # more stable than default 100
    max_depth=10,         # none let trees grow fully first
    min_samples_leaf=5,     # avoid tiny leaves
    max_features="log2",    # sqrt good with few features
    class_weight="balanced",# if classes are imbalanced
    n_jobs=-1,              # use all cores
    random_state=42

)

rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=10,
                       max_features='log2', min_samples_leaf=5,
                       n_estimators=200, n_jobs=-1, random_state=42)

In [ ]:
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.29      0.11      0.16      3083
           2       0.19      0.32      0.24      1430
           3       0.53      0.51      0.52      2903
           4       0.31      0.39      0.34      2436
           5       0.48      0.36      0.41      4177
           6       0.39      0.61      0.47      1548
           7       0.23      0.18      0.21      1897
           8       0.14      0.24      0.18      1340

    accuracy                           0.34     18814
   macro avg       0.32      0.34      0.32     18814
weighted avg       0.36      0.34      0.33     18814



Overall accuracy: 34% → Better than random guessing (12.5% for 8 classes), but still low for a production‑ready model.

Class‑level performance:

Class 3 and 6 are strongest (f1 ≈ 0.47–0.52). The model is finding signal there.

Class 1, 7, 8 are weakest (f1 ≈ 0.16–0.21). These are likely underrepresented or overlapping in feature space.

Macro vs. weighted averages: Both around 0.32–0.33, which means the imbalance isn’t the only issue — the model struggles across the board.

In [ ]:
#f this is geospatial/raster data, add neighborhood statistics (mean, variance, texture).

In [ ]:
import rasterio

with rasterio.open('/content/RGB (1).tif') as src:
    print("Number of bands:", src.count)
    print("Shape (height, width):", src.height, src.width)
    print("Data type:", src.dtypes)

    #possible multispec image

Number of bands: 4
Shape (height, width): 500 754
Data type: ('uint8', 'uint8', 'uint8', 'uint8')
